## 3.4 softmax回归

回归可以用于预测多少的问题。比如预测房屋被售出价格，或者棒球队可能获得的胜场数，又或者患者住院的天数。

事实上，我们也对分类问题感兴趣：不是问“多少”，而是问哪一个：
- 某个电子邮件是否属于垃圾邮件文件夹？
- 某个用户可能注册或不注册订阅服务？
- 某个图像描绘的是驴、狗、猫、还是鸡？
- 某人接下来最有可能看哪部电影？

通常，机器学习实践者用分类这个词描述两个有微妙差别的问题：1.我们只对样本的“硬性”类别感兴趣，即属于哪个类别；2.我们希望得到“软性”类别，即得到属于每个类别的概率。这两者的界限往往很模糊。其中的一个原因是：即使我们只关心硬类别，我们仍然使用软类别的模型。

### 3.4.1 分类问题
我们从一个图像分类问题开始。假设每次输入是一个$2 \times 2$的灰度图像。我们可以用一个标量表示每个像素值，每个图像对应四个特征$x_1, x_2, x_3, x_4$。此外，假设每个图像属于类别“猫”，“鸡”和“狗”中的一个。

接下来，我们要选择如何表示标签。我们有两个明显的选择：最直接的想法是选择$y \in \left\{1, 2, 3\right\}$，其中整数分别代表$\left\{狗,猫,鸡\right\}$。这是在计算机上存储此类信息的有效方法。如果类别间有一些自然顺序，比如说我们试图预测$\left\{婴儿,儿童,青少年,中年人,老年人\right\}$，那么这个问题转变为回归问题，并且保留这种格式是有意义的。